# OpenStreetMap Data Case Study

## Área estudada

Foi estudada a Região Metropolitana de Fortaleza, que compreende os munícipios de Fortaleza, Caucaia, Maranguape, Pacatuba e Aquiraz.

https://www.openstreetmap.org/relation/5161802#map=10/-3.9177/-38.3986

A escolha foi motivada por ser a região que vivo e portanto me interessa mais contribuir para essa região.

## Primeira Análise de Problemas no Xml

A primeira análise aconteceu no notebook test_data.ipynb

em especial nas tags foi notado somente as esperadas também foi verificado as ID e todas são únicas. Mas alguns problemas iniciais foram notados:

* No atributo 'k' das tags(a chave) foi notado inconsistências. Como: Uso incorreto das letras maiúsculas, espaços, abreviações, dentre outros casos isolados.('IBGE:GEOCODIGO')

* Também no atributo 'k' foi encontrado casos de dois pontos.(seamark:landmark:category) Definiremos que a primeira palavra diz o tipo da tag.

* Foi também visto tags com 'k'='fixme' que demandam correções. Essas serão descartadas.

* Nas ruas alguns nomes errados e abreviações.


Irei resolver apenas os problemas que influem na mudança para o SQL. Deixando para consertar nele o resto dos problemas.

<tag k="seamark:tank:category" v="tank"/>
<tag k="IBGE:GEOCODIGO" v="2307700"/>

## Nome dos atributos key nas tags

Já com os dados devidamente colocados no SQL foi análisado novamente os erros na, agora, coluna 'key'
Foi encontrado uma série de problemas nas keys das tags. como por exemplo: 'AProjetada', 'Unid.Hab', espaços dentre outros. Foram todas essas normatizadas para as letras serem sempre minúsculas e sem espaços. Com exceção de algumas siglas que ficaram maiúsculas.

Nessa Parte obtive uma certa dificuldade com os encondings. Não estava acostumado com python2 e ao utilizar as regular expressions. Houve uma certa dificuldade inicial com os acentos. Segue a função que fez o update das palavras (utilizei querys para colocar no database os novos valores)

In [1]:
#
import re
Rnormal = re.compile(r'^([\w\. _])+',re.U)
Rproblemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
Rspace = re.compile(r'^([\w\. _])*[ ]',re.U)

def update_problems_keys(name):
    '''Função para corrigir os nomes nas colunas keys'''
    if(Rproblemchars.search(name)!=None):
        if(name==u'Unid.Hab'):
            return u'unidade_habitacional'
        elif(Rspace.search(name)!=None):
            #corrigir espaços
            return Rspace.search(name).group()[:-1]+'_'+ name[(Rspace.search(name)).end():]
        else:
            print name+" Problem",Rspace.search(name)
    if(not name.islower()):
        if(name=='FID' or name=='IBGE' or name=='BR' or name=='CE'):
            return name
        elif(name==u'AProjetada'):
            return u'área_projetada'
        elif(name=='AConst'):
            return u'área_construída'
        elif(name=='GEOCODIGO'):
            return u'geocodigo'    
        elif(name=='UNIDADES_HABITACIONAIS'):
            return u'unidades_habitacionais'        
        else:
            #tornar tudo minúsculas
            return name.lower()
    return name

## Nome das ruas

Houve uma série de problemas com os nomes das ruas. Abreviações do tipo Av. ou ruas que não começavam com prefixo nenhum (Barbosa de Freitas no lugar de Avenida Barbosa de Freitas). 

Para consertar utilizei o método da lição. de criar um mapa com os prefixos comuns (Av. R. ...) e usei de expressões regulares para procura-los. Na parte de importação e alteração do banco de dados usei de dataframes do pandas dessa vez. Por motivações didáticas.

Segue o código que atualiza o nome das ruas. Foi necessário muitas buscas específicas por conta das ruas sem prefixos nenhum.

In [2]:
#nomes esperados como prefixos
expected = [u"Rua",u"Avenida",u"Beco",u"Alameda",u"Acesso",u"Travessa", u"CE",u"BR",u"4º",u'Campus',u'Praça','Vila']

#mapeamento para os nomes esperados
mapping = { u"Av.": u"Avenida",
            u"Av": u"Avenida",
            u"Tv.": u"Travessa",
            u"R.": u"Rua",
            u"Ce": u"CE",
            u"Br": u"BR"
            }
def update_problems_addr(name):
    '''Função para corrigir os nomes nas colunas value do tipo addr'''
    #torno o nome com as letras maisculuas corretas
    name=name.title()
    #inicio de busca por nomes individuais
    if(name==u'Costa Barros'):
        return u'Avenida Costa Barros'
    elif(name==u'Barbosa de Freitas'):
        return u'Avenida Barbosa de Freitas'
    elif(name==u'Joaquim Nabuco'):
        return u'Rua Joaquim Nabuco'
    elif(name==u'Eng\xb0 Sandoval S\xe1'):
        return u'Rua Engenheiro Sandoval'
    elif(name==u'2'):
        return u'Rua 2'
    elif(name==u'Alu\xedsio Mamede'):
        return u'Rua Aluísio Mamede'
    elif(name==u'Titan'):
        return u'Rua Titan'
    elif(name==u'Washington Soares'):
        return u'Avenida Washington Soares'    
    elif(name==u'José Severino'):
        return u'Rua José Severino'
    elif(name==u'Rua Nove,'):
        return u'Rua Nove'
    elif(name==u'João Paula Costa'):
        return u'Rua João Paula Costa'
    elif(name==u'Maria Teixeira Joca'):
        return u'Rua Maria Teixeira Joca'
    elif(name==u'Santa Teresinha'):
        return u'Rua Santa Teresinha'
    elif(name==u'Santa Teresinha'):
        return u'Rua Santa Teresinha'
    elif(name==u'Alphaville Fortaleza'):
        return u'Avenida Litorânea'
    #correção de prefixos
    elif(street_type_re.search(name).group() not in expected):
        rua = street_type_re.search(name)
        print name
        #aplicação do mapeamento.
        name= re.sub(street_type_re,mapping[rua.group()],name)
    return name

Foi necessário esse grande número de correções manuais por conta que não havia jeito simples de fazer o computador adivinhar se era rua ou avenida o que faltava(ou o caso do Alphaville que teve uma mudança profunda). Como eram poucos casos individuais decidi fazer na mão.

Já na função correção de prefixos tivemos coisas como R. República do líbano se tornando Rua República do líbano.

## Descartar as key='fixme'

Para Terminar a preparação de dados iniciais discartei todos os dados com o atributo key='fixme' em suas tags

In [3]:
import sqlite3
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
CHANGE="DELETE FROM ways_tags WHERE key='fixme';"
c.execute(CHANGE)
CHANGE="DELETE FROM nodes_tags WHERE key='fixme';"
c.execute(CHANGE)
db.commit()
db.close()

# Consultas e Estatísticas Básicas

## Tamanho Arquivos

Primeiro analisei o tamanho do arquivo de database e do arquivo osm original.


osm: 52MB  
database: 29MB  
ways_nodes.csv: 6MB  
ways.csv: 2MB  
ways_tags.csv: 3MB  
nodes_tags.csv: 1MB  
nodes.csv: 18MB

## Números de usuários únicos

Segue o número de usuários únicos

In [4]:
import sqlite3
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
QUERY="""SELECT  COUNT(DISTINCT e.user) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e;"""
c.execute(QUERY)
print c.fetchall()[0][0]
db.close()

501


## Número de nós e caminhos 

In [5]:
import sqlite3
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
QUERY="""SELECT COUNT(*) FROM ways;"""
c.execute(QUERY)
print "Número de ways: "+str(c.fetchall()[0][0])
QUERY="""SELECT COUNT(*) FROM nodes;"""
c.execute(QUERY)
print "Número de nodes: "+str(c.fetchall()[0][0])
db.close()

Número de ways: 44496
Número de nodes: 220814


## Análise da Culinária de Fortaleza

### Número de Nós e Ruas  Relacionados ao Tema

In [6]:
import sqlite3
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
QUERY="""SELECT COUNT(*) FROM ways_tags WHERE key='cuisine';"""
c.execute(QUERY)
print "Número no ways: "+str(c.fetchall()[0][0])
QUERY="""SELECT COUNT(*) FROM nodes_tags WHERE key='cuisine';"""
c.execute(QUERY)
print "Número no nodes: "+str(c.fetchall()[0][0])
db.close()

Número no ways: 75
Número no nodes: 68


O número de dados de restaurantes foi bem menor do que eu esperava.

### Tipos Mais Comuns de Cúlinaria

In [7]:
import sqlite3
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
QUERY="""SELECT value, COUNT(*) AS Total FROM ways_tags 
WHERE key='cuisine' GROUP BY value ORDER BY Total desc LIMIT 8;"""
c.execute(QUERY)
print "Nos ways: "
rows =c.fetchall()
for row in rows:
    print row
QUERY="""SELECT value, COUNT(*) AS Total FROM nodes_tags
WHERE key='cuisine' GROUP BY value ORDER BY Total desc LIMIT 8;"""
c.execute(QUERY)
print "Nos nós: "
rows =c.fetchall()
for row in rows:
    print row
db.close()

Nos ways: 
(u'regional', 21)
(u'burger', 14)
(u'pizza', 10)
(u'steak_house', 6)
(u'fast_food', 3)
(u'coffee_shop', 2)
(u'italian', 2)
(u'kebab', 2)
Nos nós: 
(u'pizza', 16)
(u'burger', 9)
(u'coffee_shop', 3)
(u'japanese', 3)
(u'regional', 3)
(u'ice_cream', 2)
(u'italian', 2)
(u'sandwich', 2)


## Rua Com Mais Nós

In [8]:
import sqlite3
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
QUERY="""SELECT a.value, COUNT(ways_nodes.node_id) AS Total FROM 
(SELECT id,value FROM ways_tags WHERE type='addr' AND key='street') a,ways_nodes 
WHERE a.id=ways_nodes.id GROUP BY a.value ORDER BY Total desc LIMIT 10 ;"""
c.execute(QUERY)
rows =c.fetchall()
for row in rows:
    print row
db.close()

(u'Rua Lago Azul', 599)
(u'Rua Lago Dourado', 546)
(u'Rua Passo A Passo', 399)
(u'Rua Lago Verde', 381)
(u'Rua Campo Verde', 274)
(u'Rua Gon\xe7alves Dias', 231)
(u'Avenida Dom Almeida Lustosa', 223)
(u'Avenida Washington Soares', 202)
(u'Avenida Bar\xe3o De Studart', 193)
(u'Rua Amazona', 166)


Curiosamente a rua com mais nós registrados não é uma avenida ou alguma rua de grande importância.

## Amenidades
### Número de Nós e Ruas  Relacionados ao Tema

In [9]:
import sqlite3
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
QUERY="""SELECT COUNT(*) FROM ways_tags WHERE key='amenity';"""
c.execute(QUERY)
print "Número no ways: "+str(c.fetchall()[0][0])
QUERY="""SELECT COUNT(*) FROM nodes_tags WHERE key='amenity';"""
c.execute(QUERY)
print "Número no nodes: "+str(c.fetchall()[0][0])
db.close()

Número no ways: 965
Número no nodes: 1500


Já as amenidades tiveram um bom número

### Tipos Mais Comuns de amenidades

In [10]:
import sqlite3
db=sqlite3.connect("GrandeFortaleza.db")
c=db.cursor()
QUERY="""SELECT value, COUNT(*) AS Total FROM ways_tags 
WHERE key='amenity' GROUP BY value ORDER BY Total desc LIMIT 8;"""
c.execute(QUERY)
print "Nos ways: "
rows =c.fetchall()
for row in rows:
    print row
QUERY="""SELECT value, COUNT(*) AS Total FROM nodes_tags
WHERE key='amenity' GROUP BY value ORDER BY Total desc LIMIT 8;"""
c.execute(QUERY)
print "Nos nós: "
rows =c.fetchall()
for row in rows:
    print row
db.close()

Nos ways: 
(u'parking', 231)
(u'fuel', 120)
(u'school', 93)
(u'place_of_worship', 85)
(u'restaurant', 78)
(u'fast_food', 58)
(u'bank', 44)
(u'pharmacy', 31)
Nos nós: 
(u'restaurant', 312)
(u'place_of_worship', 153)
(u'school', 138)
(u'fast_food', 113)
(u'bicycle_rental', 86)
(u'fuel', 86)
(u'pharmacy', 80)
(u'pub', 53)


Setor alimentício parece forte em fortaleza. Os restaurantes e os fast foods apareceram numa alta posição.

# Sugestões Para Melhorar a Análise dos Dados

Ainda há bastantes melhorias nos dados a serem feitas. O cep por exemplo teve ceps que vieram com hífem e outros sem hífem. Há ainda bastante limpeza e auditorias nos dados para se ter maior confiança neles.

Os dados também foram muito mais escassos do que eu esperava para uma cidade do tamanho de fortaleza. Menos de 180 entradas do tipo cuisine é baixo. Logo é necessário um maior esforço da comunidade local para contribuir com o Open Street Map para ele ser mais eficiente. Afinal somente 501 pessoas contribuiram. 

Técnicas de gamificação poderiam ser usadas para se alcançar esse objetívo. Usando-se de rankings e outras ferramentas competitivas. quanto mais contribuir maior o level da pessoa na comunidade por exemplo. 
 * Benefícios:  
   - Aumento engajamento  
 * Problemas esperados:  
   - O engajamento precisa de incentivos iniciais

Algum tipo de vínculo com instituições locais para se conseguir descontos/benefícios para grandes contribuidores seria o ideal.
 * Benefícios:  
   - Grande aumento do engajamento  
 * Problemas esperados:  
   - Pode ser necessário aporte financeiro
Poderia-se convocar os já apoiadores do projeto locais para fazer esse diálogo.

Também seria bom cruzar os dados com a API do google. Para assim validar os dados e talvez suprir dados ausentes e defeituosos
 * Benefícios:  
   - Grande aumento da validade  
 * Problemas esperados:  
   - Os dados da google também podem conter erros

# Conclusão

Foi um trabalho interessante porém cansativo pelo fato da minha total inexpêriencia prévia a lição com todas as ferramentas utilizadas. As ruas acredito que consegui limpar razoavelmente bem. Análises foram feitas e foi notado a deficiência em dados e contribuidores na minha região.


Uma extensão que acharia interessante seria analisar essas 501 pessoas e verificar quanto cada um contribuiu utilizando de testes estatísticos. Como é algo similar ao sample_project decidi concentrar em outras tarefas.